# Módulo destinado a geração dos arquivos parquet
##  O módulo deve ser executado para a preparação dos dados previamente ao uso do processo investigativo.
1. Passo - Gera embeddings do Nome Fantasia 

In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, from_json, substring, when, length, split, lit
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
from trata_dados import trata_empresa
from trata_dados import trata_socio
from trata_dados import monta_vertice

In [3]:
# Nomes de aquivos e pastas
# Arquivos de entrada
csv_pj = 'dataset/completo/empresas_estabelecimentos*.csv'
csv_socios = 'dataset/completo/socios*.csv' 
parquet_dir_name = 'output/embeddings_nome'
# Arquivos de saída
parquet_nodes = 'dataset/grapho/nodes'
parquet_edges = 'dataset/grapho/edges'

In [4]:
spark = SparkSession.builder \
        .config("spark.driver.memory", "128g") \
        .appName("gera_embedding_nome") \
        .getOrCreate()

24/08/22 13:39:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
empresas = trata_empresa(csv_pj, spark)


In [6]:
# Gera dataframe com nome fantasia
nomes_empresas = empresas.select(col('id'),
                      col('te_dados_es.nomeFantasia').alias('nomeFantasia')) \
                     .filter(col('te_dados_es.nomeFantasia') != '')

#### Tratamento Nome Fantasia

In [7]:
from sentence_transformers import SentenceTransformer
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# Função para criar embeddings usando SentenceTransformer
def generate_embeddings_with_model(text, model):
    embeddings = model.encode(text)
    return embeddings.tolist()

# Criando a UDF
# Criando o modelo SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Criando a UDF com o modelo como parâmetro
generate_embeddings_udf = udf(lambda text: generate_embeddings_with_model(text, model), ArrayType(FloatType()))

In [8]:
# Gera embeddings
nomes_empresas = nomes_empresas.withColumn('embeddings', generate_embeddings_udf(col('nomeFantasia')))

In [9]:
# Persista o DataFrame para consulta posterior
nomes_empresas.write.mode('overwrite').parquet(parquet_dir_name)

24/08/21 19:01:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


#### Tratamento Nós e Arestas

In [7]:
socios = trata_socio(csv_socios, spark)

In [8]:
nodes = monta_vertice(empresas, socios)

In [9]:
socios.write.mode('overwrite').parquet(parquet_edges)
nodes.write.mode('overwrite').parquet(parquet_nodes)

24/08/22 13:47:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
spark.stop()